In [1]:
import os
gpu_num = 0 # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Colab does currently not support the latest version of ipython.
# Thus, the preview does not work in Colab. However, whenever possible we
# strongly recommend to use the scene preview mode.
try: # detect if the notebook runs in Colab
    import google.colab
    colab_compat = True # deactivate preview
except:
    colab_compat = False
resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [3]:
# Load integrated scene
scene = load_scene('NSYSU/xml/NSYSU2.xml')
scene.frequency = 3.66e9
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [4]:
obj = scene.get("map_4_osm_buildings-wall") 
obj.radio_material = "itu_concrete"

In [5]:
# Select an example object from the scene
so = scene.get("itu_concrete")

# Print name of assigned radio material for different frequenies
for f in [3.5e9, 2.14e9]: # Print for differrent frequencies
    scene.frequency = f
    print(f"\nRadioMaterial: {so.name} @ {scene.frequency/1e9:.2f}GHz")
    print("Conductivity:", so.conductivity.numpy())
    print("Relative permittivity:", so.relative_permittivity.numpy())
    print("Complex relative permittivity:", so.complex_relative_permittivity.numpy())
    print("Relative permeability:", so.relative_permeability.numpy())
    print("Scattering coefficient:", so.scattering_coefficient.numpy())
    print("XPD coefficient:", so.xpd_coefficient.numpy())


RadioMaterial: itu_concrete @ 3.50GHz
Conductivity: 0.123086944
Relative permittivity: 5.24
Complex relative permittivity: (5.24-0.63214296j)
Relative permeability: 1.0
Scattering coefficient: 0.0
XPD coefficient: 0.0

RadioMaterial: itu_concrete @ 2.14GHz
Conductivity: 0.0837706
Relative permittivity: 5.24
Complex relative permittivity: (5.24-0.7036379j)
Relative permeability: 1.0
Scattering coefficient: 0.0
XPD coefficient: 0.0


In [6]:
# change scattering coefficient
# check it using the above cell

import tensorflow as tf
so.scattering_coefficient = tf.Variable(0.1, dtype=tf.float32)

In [7]:
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Create transmitter
tx = Transmitter(name="tx",
                 position=[149.5, -97.1, 26],
                 orientation=[0, 0, 0])

Tx_pos = np.array([149.5, -97.1, 26]).reshape(-1, 3)

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
              position=[147.3, 124, 1.5],
              orientation=[0,0,0])

# Add receiver instance to scene
scene.add(rx)

scene.synthetic_array = True

tx.look_at(rx) # Transmitter points towards receiver

scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…

In [8]:
cm = scene.coverage_map(max_depth=8, 
                        los=True, 
                        reflection=True, 
                        diffraction=True, 
                        check_scene=False) 

In [9]:
min_gain_db = -160 # in dB; ignore any position with less than -130 dB path gain
# max_gain_db = 0 # in dB; ignore strong paths

# sample points in a 5-400m radius around the receiver
min_dist = 0 # in m
max_dist = 1000 # in m

batch_size = 10000

#sample batch_size random user positions from coverage map
ue_pos = cm.sample_positions(batch_size=batch_size,
                             min_gain_db=min_gain_db,
                             # max_gain_db=max_gain_db,
                             min_dist=min_dist,
                             max_dist=max_dist)

position_file_name = 'UE_pos.txt'
with open(position_file_name, 'w') as file:
    num = 0
    for item in ue_pos:
        file.write(f"{item}\n")
        num += 1
        
position_file_name = 'UE_pos_with_index.txt'
with open(position_file_name, 'w') as file:
    num = 0
    for item in ue_pos:
        file.write(f"{num} : {item}\n")
        num += 1

In [10]:
# Open the file and read its contents
with open('UE_pos.txt', 'r') as file:
    lines = file.readlines()

# Initialize an empty list to store the coordinates
ue_pos = []

# Loop through each line in the file
for line in lines:
    # Extract the numbers inside the brackets and convert them to floats
    coord = [float(num) for num in line.strip().strip('[]').split()]
    # Add the coordinate to the list
    ue_pos.append(coord)

# Now, coordinates contains all the coordinates as lists of floats
print(ue_pos)
print(len(ue_pos))

batch_size = len(ue_pos)


[[385.6337, 420.09857, 1.5], [385.6105, 382.04907, 1.5], [352.9619, 83.24408, 1.5], [550.2628, 214.48055, 1.5], [401.89047, -267.41766, 1.5], [-430.72354, 247.90654, 1.5], [475.26974, 496.61505, 1.5], [-528.18494, 81.34434, 1.5], [15.973072, 563.4723, 1.5], [-540.7025, 244.93094, 1.5], [-346.92584, 293.31226, 1.5], [303.02893, -33.825386, 1.5], [-542.303, 309.9058, 1.5], [-308.8594, 528.9505, 1.5], [-418.3175, -36.872566, 1.5], [-234.08015, 124.56995, 1.5], [-306.393, -354.40683, 1.5], [566.1087, -81.94637, 1.5], [-209.63593, -555.92773, 1.5], [-326.74036, -142.5504, 1.5], [535.6991, 168.73984, 1.5], [474.89062, 590.8217, 1.5], [369.50055, 151.66383, 1.5], [-516.14264, -46.947853, 1.5], [35.9316, -504.844, 1.5], [553.56226, -245.58934, 1.5], [-487.69556, -153.57953, 1.5], [-177.55522, 450.13348, 1.5], [493.68112, 286.86246, 1.5], [154.71838, -307.6333, 1.5], [25.006653, 201.82536, 1.5], [277.31302, -570.9564, 1.5], [-356.93436, -391.30298, 1.5], [-310.54926, 13.525138, 1.5], [451.18872

In [13]:
import scipy.io as sio

# Remove old receivers from scene
scene.remove("rx")
for i in range(batch_size):
    scene.remove(f"rx-{i}")

# Configure antenna array for all receivers (=UEs)
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1, # Each receiver is equipped with 4 tx antennas (uplink)
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso", # UE orientation is random
                             polarization="V")

# Create batch_size receivers
# all_paths = []

UE_num = 0
UE_position_list = []
for i in range(10):
    rx = Receiver(name=f"rx-{i}",
                  position=ue_pos[i], # Random position sampled from coverage map
                  )
    scene.add(rx)
    
    paths = scene.compute_paths(max_depth=8,
                            method='fibonacci',
                            num_samples=1e6,
                            los = True,
                            reflection=True,
                            diffraction=True,
                            check_scene=True,
                            scattering=True,
                            edge_diffraction=True,
                            scat_keep_prob=0.01)
    
    ray_num = len(paths.types.numpy()[0])
    print(f"ray number : {ray_num}")
                  
    path_gain_list = []
    path_phase_list = []
    path_delay_list = []
    path_AOD_hor_list = []
    path_AOD_ver_list = []
    path_AOA_hor_list = []
    path_AOA_ver_list = []
    
    count = 0
    for j in range(ray_num):
        path_idx = j 
        # For a detailed overview of the dimensions of all properties, have a look at the API documentation
        # print(f"\n--- Detailed results for path {path_idx} ---")

        # print(f"Channel coefficient: {paths.a[0,0,0,0,0,path_idx, 0].numpy()}")
        path_coefficients = paths.a[0,0,0,0,0,path_idx, 0].numpy()
        if path_coefficients == 0:
            continue
        path_gain = np.abs(path_coefficients)
        path_phase = np.angle(path_coefficients)
        path_gain_list.append(path_gain)
        path_phase_list.append(path_phase)

        # print(f"Propagation delay: {paths.tau[0,0,0,path_idx].numpy()*1e6:.8f} us")
        path_delay = paths.tau[0,0,0,path_idx].numpy()
        path_delay_list.append(path_delay)

        # print(f"Zenith angle of departure: {paths.theta_t[0,0,0,path_idx]:.4f} rad")
        path_AOD_ver = paths.theta_t[0,0,0,path_idx]
        path_AOD_ver_list.append(path_AOD_ver)

        # print(f"Azimuth angle of departure: {paths.phi_t[0,0,0,path_idx]:.4f} rad")
        path_AOD_hor = paths.phi_t[0,0,0,path_idx]
        path_AOD_hor_list.append(path_AOD_hor)

        # print(f"Zenith angle of arrival: {paths.theta_r[0,0,0,path_idx]:.4f} rad")
        path_AOA_ver = paths.theta_r[0,0,0,path_idx]
        path_AOA_ver_list.append(path_AOA_ver)

        # print(f"Azimuth angle of arrival: {paths.phi_r[0,0,0,path_idx]:.4f} rad")
        path_AOA_hor = paths.phi_r[0,0,0,path_idx]
        path_AOA_hor_list.append(path_AOA_hor)
        count += 1
        if count >= 297:
            break
    k = len(path_gain_list)    
        
    def pad_to_297(arr_list):
        arr = np.array(arr_list).reshape(-1, 1)
        return np.pad(arr, ((0, 297 - arr.shape[0]), (0, 0)), 'constant', constant_values=0)

    path_gains = pad_to_297(path_gain_list)
    path_phases = pad_to_297(path_phase_list)
    path_delays = pad_to_297(path_delay_list)
    path_AOA_hors = pad_to_297(path_AOA_hor_list)
    path_AOA_vers = pad_to_297(path_AOA_ver_list)
    path_AOD_hors = pad_to_297(path_AOD_hor_list)
    path_AOD_vers = pad_to_297(path_AOD_ver_list)

    if np.all(path_gains == 0):
        scene.remove(f"rx-{i}")
        print(f"skip data{i}")  
        continue
    
    else:
        UE_position_list.append(ue_pos[i])
        UE_postion_arr = np.array(UE_position_list).reshape(-1, 3)
        UE_num += 1
        
        data_dict = {
            'path_gain': path_gains,
            'path_phase': path_phases,
            'path_delay': path_delays,
            'path_AOA_hor': path_AOA_hors,
            'path_AOA_ver': path_AOA_vers,
            'path_AOD_hor': path_AOD_hors,
            'path_AOD_ver': path_AOD_vers,
            }

        mat_struct = {'sim': data_dict}
    
        sio.savemat(f'MAT_SISO/data{i}.mat', mat_struct)

        print(f"data{i} file created successfully.")
        # print("##############################################")
        scene.remove(f"rx-{i}")
        # print(f"rx-{i} removed")
        
pos_dict = {
    'agent_num' : UE_num,
    'agent' : UE_postion_arr,
    'anchor_num' : 1,
    'anchor' : Tx_pos
}
    
sio.savemat(f'TxRx_pos/position.mat', pos_dict)



ray number : 400
data0 file created successfully.
ray number : 392
data1 file created successfully.
ray number : 376
data2 file created successfully.
ray number : 279
data3 file created successfully.
ray number : 64
data4 file created successfully.
ray number : 207
data5 file created successfully.
ray number : 326
data6 file created successfully.
ray number : 376
data7 file created successfully.
ray number : 326
data8 file created successfully.
ray number : 269
data9 file created successfully.


In [ ]:
# Remove position
import os

# Paths to your files
txt_file_path = 'UE_pos_with_index.txt'
mat_files_folder = 'MAT_data_4T4R'

# Read the indices from the .txt file
with open(txt_file_path, 'r') as file:
    lines = file.readlines()

# Extract indices from the lines
indices = [int(line.split(" :")[0].strip()) for line in lines]

# Get a list of existing .mat files in the folder
existing_mat_files = [f for f in os.listdir(mat_files_folder) if f.endswith('.mat')]

# Determine missing indices
missing_indices = []
for idx in indices:
    if f"data{idx}.mat" not in existing_mat_files:
        missing_indices.append(idx)

# Filter out lines corresponding to missing indices
filtered_lines = [line for line in lines if int(line.split(" :")[0].strip()) not in missing_indices]

# Write the filtered lines back to the .txt file (or a new file if you prefer)
with open('filtered_UE_pos_with_index.txt', 'w') as file:
    file.writelines(filtered_lines)

print(f"Removed {len(missing_indices)} entries corresponding to missing .mat files.")
